In [1]:
import pandas as pd
import numpy as np
import datetime
import glob
import os
from google.colab import drive

In [64]:
# Mount Google Drive
drive.mount('/content/drive')
# Path to the directory containing folders for each year in Google Drive
drive_dir = '/content/drive/MyDrive/bhavcopy/Dataset - 2018-2022/'
# Get a list of all year folders
year_folders = [folder for folder in os.listdir(drive_dir) if os.path.isdir(os.path.join(drive_dir, folder))]
year_folders.sort() #['2018', '2019', '2020', '2021', '2022']

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
file_set = set()
from datetime import datetime
for year_folder in year_folders:
    # Get a list of all CSV files in the year's folder with the format cm.csv
    file_list = glob.glob(drive_dir + year_folder + '/cm*.csv')
  # Extract and convert dates from file names
    file_dates = [datetime.strptime(os.path.basename(file_path)[2:-8], '%d%b%Y') for file_path in file_list]

    # Sort files by date
    file_list_sorted = [file_path for _, file_path in sorted(zip(file_dates, file_list))]

    # Process each CSV file and adding to a list
    for file_path in file_list_sorted:
      file_set.add(file_path)
month = {1:'JAN', 2:'FEB', 3:'MAR', 4:'APR', 5:'MAY', 6:'JUN', 7:'JUL', 8:'AUG', 9:'SEP', 10:'OCT', 11:'NOV', 12:'DEC'}

In [5]:
def file_format(s_date):
  v_date = drive_dir + str(s_date.year) + '/cm'+ str(s_date.day).zfill(2) + month[s_date.month] + str(s_date.year) +'bhav.csv'
  return v_date

def valid_date(date1, date2):
  start_file = ""
  while(date1>date2):
    start_file = file_format(date1)
    if(start_file in file_set):
      break
    date1 = date1 - datetime.timedelta(days=1)
  return start_file, date1

In [6]:
# Initialize an empty DataFrame
final_df = pd.DataFrame(columns=['SYMBOL'])
############################################################################Function to calculate weekly returns################################################
def calculate_weekly_return(file1, file2, week_number, date1, date2):
    global final_df  # Use global final_df for modification

    # Load CSV file into DataFrame
    data1 = pd.read_csv(file1)
    data2 = pd.read_csv(file2)

    # Filter data to include only series 'EQ'
    data1 = data1[data1['SERIES'] == 'EQ']
    data2 = data2[data2['SERIES'] == 'EQ']

    # Concatenate SYMBOL and SERIES to create a unique identifier
    data1['UNIQUE_ID'] = data1['SYMBOL'] + '_' + data1['SERIES']
    data2['UNIQUE_ID'] = data2['SYMBOL'] + '_' + data2['SERIES']

    # Set 'UNIQUE_ID' as the index for both DataFrames
    data1.set_index('UNIQUE_ID', inplace=True)
    data2.set_index('UNIQUE_ID', inplace=True)

    # Find common symbols
    common_symbols = data1.index.intersection(data2.index)

    # Iterate over common symbols
    for symbol in common_symbols:
        closing_price_file1 = data1.loc[symbol, 'CLOSE']
        closing_price_file2 = data2.loc[symbol, 'CLOSE']
        ret = (closing_price_file1 - closing_price_file2) * 100 / closing_price_file2
        week_column_name = f'Week{week_number} ' + f'({date1}' + f' to{date2})'
        # Check if symbol already in final_df
        if symbol in final_df.index:
            final_df.loc[symbol, week_column_name] = ret
        else:
            new_row = pd.DataFrame({week_column_name: [ret]}, index=[symbol])
            final_df = pd.concat([final_df, new_row])

    # Save to CSV
    final_df.to_csv('weekly_return_values.csv', index=True)

In [9]:
from dateutil.relativedelta import relativedelta
import datetime
# Define the start and end dates for the backtesting period
start_date = datetime.date(2018, 1, 14)
end_date = datetime.date(2023, 1, 1)  # 4-year period

i=1
while(end_date >= start_date):
  # print("Week: ",i)
  prior_start_week = start_date - datetime.timedelta(days=7)
  file1, date1= valid_date(start_date, prior_start_week)
  # Calculate 1 week back
  prior_to_prior_end_week = prior_start_week - datetime.timedelta(days=7)
  file2, date2 = valid_date(prior_start_week, prior_to_prior_end_week )
  # print(file1, file2)
  calculate_weekly_return(file1, file2,i, date1, date2)
  i = i + 1
  start_date = start_date + datetime.timedelta(days=7)
print(i)


In [13]:
rebalancing_date = datetime.date(2021, 2, 1) #7 Feb 2021 - 1st week of Feb'21
One_month_before_reb_date = rebalancing_date - relativedelta(months=1)
temp_date_one = One_month_before_reb_date - datetime.timedelta(days=7)
price_file_one_month, closing_WeekDate_one_month= valid_date(One_month_before_reb_date, temp_date_one)

Seven_month_before_reb_date = rebalancing_date - relativedelta(months=7)
temp_date_two = Seven_month_before_reb_date - datetime.timedelta(days=7)
price_file_seven_month, closing_WeekDate_seven_month= valid_date(Seven_month_before_reb_date, temp_date_two)

Thirteen_month_before_reb_date = rebalancing_date - relativedelta(months=13)
temp_date_three = Thirteen_month_before_reb_date - datetime.timedelta(days=7)
price_file_thirteen_month, closing_WeekDate_thirteen_month= valid_date(Thirteen_month_before_reb_date, temp_date_three)


In [14]:
print(price_file_one_month, closing_WeekDate_one_month)
print(price_file_seven_month, closing_WeekDate_seven_month)
print(price_file_thirteen_month, closing_WeekDate_thirteen_month)

/content/drive/MyDrive/bhavcopy/Dataset - 2018-2022/2021/cm01JAN2021bhav.csv 2021-01-01
/content/drive/MyDrive/bhavcopy/Dataset - 2018-2022/2020/cm01JUL2020bhav.csv 2020-07-01
/content/drive/MyDrive/bhavcopy/Dataset - 2018-2022/2020/cm01JAN2020bhav.csv 2020-01-01


In [78]:
final_df_one = pd.DataFrame(columns=['SYMBOL', '6-mo Price Momentum', '12-mo Price Momentum'])
list_of_valid_symbols = []  # which have at least 6-month price momentum history
risk_free_rate = 0  # can be modified accordingly later

def calculate_price_momentum_6mo(file1, file2):
    global final_df_one  # Use global final_df for modification
    global list_of_valid_symbols

    # Load CSV file into DataFrame
    data1 = pd.read_csv(file1)
    data2 = pd.read_csv(file2)

    # Filter data to include only series 'EQ'
    data1 = data1[data1['SERIES'] == 'EQ']
    data2 = data2[data2['SERIES'] == 'EQ']

    # Concatenate SYMBOL and SERIES to create a unique identifier
    data1['UNIQUE_ID'] = data1['SYMBOL'] + '_' + data1['SERIES']
    data2['UNIQUE_ID'] = data2['SYMBOL'] + '_' + data2['SERIES']

    # Set 'UNIQUE_ID' as the index for both DataFrames
    data1.set_index('UNIQUE_ID', inplace=True)
    data2.set_index('UNIQUE_ID', inplace=True)

    # Find common symbols
    common_symbols = data1.index.intersection(data2.index)
    list_of_valid_symbols = common_symbols.tolist()

    # Iterate over common symbols
    for symbol in common_symbols:
        closing_price_file1 = data1.loc[symbol, 'CLOSE']
        closing_price_file2 = data2.loc[symbol, 'CLOSE']
        ret = (closing_price_file1/closing_price_file2) - 1 - risk_free_rate
        column_name = "6-mo Price Momentum"

        if symbol in final_df_one['SYMBOL'].values:
            final_df_one.loc[final_df_one['SYMBOL'] == symbol, column_name] = ret
        else:
            new_row = pd.DataFrame({'SYMBOL': [symbol], column_name: [ret]})
            final_df_one = pd.concat([final_df_one, new_row], ignore_index=True)

def calculate_price_momentum_12mo(file1, file2):
    global final_df_one  # Use global final_df for modification

    # Load CSV file into DataFrame
    data1 = pd.read_csv(file1)
    data2 = pd.read_csv(file2)

    # Filter data to include only series 'EQ'
    data1 = data1[data1['SERIES'] == 'EQ']
    data2 = data2[data2['SERIES'] == 'EQ']

    # Concatenate SYMBOL and SERIES to create a unique identifier
    data1['UNIQUE_ID'] = data1['SYMBOL'] + '_' + data1['SERIES']
    data2['UNIQUE_ID'] = data2['SYMBOL'] + '_' + data2['SERIES']

    # Set 'UNIQUE_ID' as the index for both DataFrames
    data1.set_index('UNIQUE_ID', inplace=True)
    data2.set_index('UNIQUE_ID', inplace=True)

    common_symbols = pd.Index(list_of_valid_symbols).intersection(data2.index)

    # Iterate over common symbols
    for symbol in common_symbols:
        closing_price_file1 = data1.loc[symbol, 'CLOSE']
        closing_price_file2 = data2.loc[symbol, 'CLOSE']
        ret = (closing_price_file1/closing_price_file2) - 1 - risk_free_rate
        column_name = "12-mo Price Momentum"

        if symbol in final_df_one['SYMBOL'].values:
            final_df_one.loc[final_df_one['SYMBOL'] == symbol, column_name] = ret
        else:
            new_row = pd.DataFrame({'SYMBOL': [symbol], column_name: [ret]})
            final_df_one = pd.concat([final_df_one, new_row], ignore_index=True)
    # # Save to CSV
    # final_df_one.to_csv('weekly_return_values.csv', index=True)

In [79]:
calculate_price_momentum_6mo(price_file_one_month, price_file_seven_month)
calculate_price_momentum_12mo(price_file_one_month, price_file_thirteen_month)

In [80]:
print(final_df_one)

             SYMBOL  6-mo Price Momentum 12-mo Price Momentum
0      20MICRONS_EQ             0.232308             0.118715
1     21STCENMGM_EQ            -0.040541                  NaN
2     3IINFOTECH_EQ             1.410714                2.375
3        3MINDIA_EQ             0.135563            -0.027362
4         3PLAND_EQ             0.736434             0.866667
...             ...                  ...                  ...
1314   ZODJRDMKJ_EQ             0.180894            -0.093604
1315        ZOTA_EQ             0.250351            -0.055438
1316       ZUARI_EQ            -0.019539             0.063587
1317   ZUARIGLOB_EQ             0.358095             0.470103
1318   ZYDUSWELL_EQ             0.616569             0.406937

[1319 rows x 3 columns]


In [93]:
import math
def risk_adjusted_price_momentum():
  weekly_return_file = drive_dir + 'weekly_return_values.csv'
  df = pd.read_csv(weekly_return_file)
  global list_of_valid_symbols
  global final_df_one
  no_of_weeks = 156
  # Extract column headings
  row_headings = df['SYMBOL']
  column_headings = df.columns.tolist()
  for symbol in list_of_valid_symbols:
    # Get the row for the specified stock
    stock_row = df[df['SYMBOL'] == symbol].reset_index(drop=True)
    stock_row = stock_row.iloc[:, 1:no_of_weeks + 1]
    # Calculate the standard deviation for the row
    std_dev_weekly = stock_row.std(axis=1).values[0]
    std_dev_annual = std_dev_weekly*math.sqrt(52)

    column_name = "annualized std deviation of weekly returns"

    # Check if the column exists in final_df_one, if not, create it
    if column_name not in final_df_one.columns:
        final_df_one[column_name] = pd.NA

    # Update the standard deviation value for the symbol in final_df_one
    final_df_one.loc[final_df_one['SYMBOL'] == symbol, column_name] = std_dev_annual

risk_adjusted_price_momentum()

In [94]:
print(final_df_one)

             SYMBOL  6-mo Price Momentum 12-mo Price Momentum  \
0      20MICRONS_EQ             0.232308             0.118715   
1     21STCENMGM_EQ            -0.040541                  NaN   
2     3IINFOTECH_EQ             1.410714                2.375   
3        3MINDIA_EQ             0.135563            -0.027362   
4         3PLAND_EQ             0.736434             0.866667   
...             ...                  ...                  ...   
1314   ZODJRDMKJ_EQ             0.180894            -0.093604   
1315        ZOTA_EQ             0.250351            -0.055438   
1316       ZUARI_EQ            -0.019539             0.063587   
1317   ZUARIGLOB_EQ             0.358095             0.470103   
1318   ZYDUSWELL_EQ             0.616569             0.406937   

     annualized std deviation of weekly returns  
0                                     42.979822  
1                                     31.358179  
2                                     69.664465  
3                  

In [95]:
final_df_one['6 mo Momentum/Std Dev'] = final_df_one['6-mo Price Momentum'] / final_df_one['annualized std deviation of weekly returns']
final_df_one['12 mo Momentum/Std Dev'] = final_df_one['12-mo Price Momentum'] / final_df_one['annualized std deviation of weekly returns']

In [96]:
print(final_df_one)

             SYMBOL  6-mo Price Momentum 12-mo Price Momentum  \
0      20MICRONS_EQ             0.232308             0.118715   
1     21STCENMGM_EQ            -0.040541                  NaN   
2     3IINFOTECH_EQ             1.410714                2.375   
3        3MINDIA_EQ             0.135563            -0.027362   
4         3PLAND_EQ             0.736434             0.866667   
...             ...                  ...                  ...   
1314   ZODJRDMKJ_EQ             0.180894            -0.093604   
1315        ZOTA_EQ             0.250351            -0.055438   
1316       ZUARI_EQ            -0.019539             0.063587   
1317   ZUARIGLOB_EQ             0.358095             0.470103   
1318   ZYDUSWELL_EQ             0.616569             0.406937   

     annualized std deviation of weekly returns 6 mo Momentum/Std Dev  \
0                                     42.979822              0.005405   
1                                     31.358179             -0.001293   


In [99]:
# Calculate mean and standard deviation ignoring NaN values
mean_6mo = final_df_one['6 mo Momentum/Std Dev'].mean(skipna=True)
std_6mo = final_df_one['6 mo Momentum/Std Dev'].std(skipna=True)

mean_12mo = final_df_one['12 mo Momentum/Std Dev'].mean(skipna=True)
std_12mo = final_df_one['12 mo Momentum/Std Dev'].std(skipna=True)

# Calculate z-scores
final_df_one['6-mo Price Momentum Zscore'] = (final_df_one['6 mo Momentum/Std Dev'] - mean_6mo) / std_6mo
final_df_one['12-mo Price Momentum Zscore'] = (final_df_one['12 mo Momentum/Std Dev'] - mean_12mo) / std_12mo

In [100]:
print(final_df_one)

             SYMBOL  6-mo Price Momentum 12-mo Price Momentum  \
0      20MICRONS_EQ             0.232308             0.118715   
1     21STCENMGM_EQ            -0.040541                  NaN   
2     3IINFOTECH_EQ             1.410714                2.375   
3        3MINDIA_EQ             0.135563            -0.027362   
4         3PLAND_EQ             0.736434             0.866667   
...             ...                  ...                  ...   
1314   ZODJRDMKJ_EQ             0.180894            -0.093604   
1315        ZOTA_EQ             0.250351            -0.055438   
1316       ZUARI_EQ            -0.019539             0.063587   
1317   ZUARIGLOB_EQ             0.358095             0.470103   
1318   ZYDUSWELL_EQ             0.616569             0.406937   

     annualized std deviation of weekly returns 6 mo Momentum/Std Dev  \
0                                     42.979822              0.005405   
1                                     31.358179             -0.001293   


In [104]:
#Calculating single Momentum combined score (C)
weight_6mo = 0.5 #Can be customized later
weight_12mo = 0.5 #Can be customized later
final_df_one['C'] =  final_df_one['6-mo Price Momentum Zscore']*weight_6mo + final_df_one['12-mo Price Momentum Zscore'].fillna(0)*weight_12mo

In [105]:
print(final_df_one)

             SYMBOL  6-mo Price Momentum 12-mo Price Momentum  \
0      20MICRONS_EQ             0.232308             0.118715   
1     21STCENMGM_EQ            -0.040541                  NaN   
2     3IINFOTECH_EQ             1.410714                2.375   
3        3MINDIA_EQ             0.135563            -0.027362   
4         3PLAND_EQ             0.736434             0.866667   
...             ...                  ...                  ...   
1314   ZODJRDMKJ_EQ             0.180894            -0.093604   
1315        ZOTA_EQ             0.250351            -0.055438   
1316       ZUARI_EQ            -0.019539             0.063587   
1317   ZUARIGLOB_EQ             0.358095             0.470103   
1318   ZYDUSWELL_EQ             0.616569             0.406937   

     annualized std deviation of weekly returns 6 mo Momentum/Std Dev  \
0                                     42.979822              0.005405   
1                                     31.358179             -0.001293   


In [115]:
# Calculating standardized momentum Z-score(Z)
mean_C_val = final_df_one['C'].mean(skipna=True)
std_C_val = final_df_one['C'].std(skipna=True)

# Calculate z-scores using pandas
final_df_one['Z'] = (final_df_one['C'] - mean_C_val) / std_C_val

# Winsorizing Z-score to limits of +/-3
final_df_one['Z'] = final_df_one['Z'].clip(-3, 3)

final_df_one['momentum score'] = np.where(final_df_one['Z'] >= 0, 1 + final_df_one['Z'], 1 / (1 - final_df_one['Z']))

In [116]:
print(final_df_one)

             SYMBOL  6-mo Price Momentum 12-mo Price Momentum  \
0      20MICRONS_EQ             0.232308             0.118715   
1     21STCENMGM_EQ            -0.040541                  NaN   
2     3IINFOTECH_EQ             1.410714                2.375   
3        3MINDIA_EQ             0.135563            -0.027362   
4         3PLAND_EQ             0.736434             0.866667   
...             ...                  ...                  ...   
1314   ZODJRDMKJ_EQ             0.180894            -0.093604   
1315        ZOTA_EQ             0.250351            -0.055438   
1316       ZUARI_EQ            -0.019539             0.063587   
1317   ZUARIGLOB_EQ             0.358095             0.470103   
1318   ZYDUSWELL_EQ             0.616569             0.406937   

     annualized std deviation of weekly returns 6 mo Momentum/Std Dev  \
0                                     42.979822              0.005405   
1                                     31.358179             -0.001293   


In [118]:

data1 = pd.read_csv(price_file_one_month)
# Filter data to include only series 'EQ'
data1 = data1[data1['SERIES'] == 'EQ']
# Concatenate SYMBOL and SERIES to create a unique identifier
data1['SYMBOL'] = data1['SYMBOL'] + '_' + data1['SERIES']
# Set 'SYMBOL' as the index for both DataFrames
data1.set_index('SYMBOL', inplace=True)

# Sort final_df_one by 'Z' (descending) and then by 'TOTTRDVAL' (descending)
final_df_one_sorted = final_df_one.merge(data1, on='SYMBOL')
final_df_one_sorted = final_df_one_sorted[['SYMBOL', 'momentum score', 'TOTTRDVAL']].sort_values(by=['momentum score', 'TOTTRDVAL'], ascending=[False, False]).reset_index(drop=True)

# Create a copy of final_df_one_sorted with only 'SYMBOL', 'Z', and 'TOTTRDQTY' columns
final_stock_rankings = final_df_one_sorted[['SYMBOL', 'momentum score', 'TOTTRDVAL']].copy()

# Print final_df_two to verify
print(final_stock_rankings)

             SYMBOL momentum score     TOTTRDVAL
0      INDIAMART_EQ              4  3.672267e+09
1         MASTEK_EQ              4  5.733329e+08
2     ADANIGREEN_EQ              4  3.169779e+08
3          TANLA_EQ              4  2.690557e+08
4           CDSL_EQ              4  2.236121e+08
...             ...            ...           ...
1314     JUMPNET_EQ       0.329574  2.218554e+07
1315     MAJESCO_EQ       0.312385  1.394496e+05
1316   EICHERMOT_EQ       0.301203  2.318286e+09
1317    AXISGOLD_EQ           0.25  4.616323e+06
1318   AXISNIFTY_EQ           0.25  5.897292e+05

[1319 rows x 3 columns]


In [119]:
final_stock_rankings.to_csv('final_stock_rankings.csv', index=True)